In [92]:
## Hidden Markov Model
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('aggregated.csv')
trend_data = df['nvidia'].values
price_data = df['NVDA Monthly'].values
random.seed(42)

## Hidden Markov Model
Using Bauch as our maximizer and Viterbi to decide states, we draft our plan as such: \

1. q_state = 3 { Buy, Hold, Sell }

2. hidden_states = 

3. transition matrix is defined as: \
    B>B(signal=2), B>H(signal=0), B>S(signal=-2), \
    H>B(signal=1), H>H(signal=0), H>S(signal=-1), \
    S>B(signal=2), S>H(signal=0), S>S(signal=-2)

4. emission matrix is defined as "Given state i, P(j)": \
    Up (1) = P(B|S|H), \
    Down (0) = P(B|S|H), \
    Stale (2) = P(B|S|H)
    

#### Matrix plan v1
Transition matrix:\


Emission matrix:\
if -0.1 < ROC < 0.1 : Stale \
if ROC > 0.1 : Up \
if ROC < -0.1 : Down




In [93]:
define_count = {}

for obs in observations:
    if obs in define_count:
        define_count[obs] += 1
    else:
        define_count[obs] = 1

In [94]:
n_hidden = 3 # states: buy, hold, sell
n_obs = 3 # observations: up, down, no change
transition_matrix = np.array([[0.1, 0.2, 0.3],
                              [0.1, 0.2, 0.3],
                              [0.1, 0.2, 0.3]])
emission_matrix = np.array([[0.1, 0.2, 0.3],
                            [0.1, 0.2, 0.3],
                            [0.1, 0.2, 0.3]])
                            

In [95]:
### TESTING ###
threshold = 0.05
total_trend, total_price = len(trend_data), len(price_data)

discrete_trend_data = [0 if x <= -threshold else 1 if x >= threshold else 2 for x in trend_data]
discrete_price_data = [0 if x <= -threshold else 1 if x >= threshold else 2 for x in price_data]


# Counting the number of symbols
price_symbols = {'sell': 0, 'buy': 0, 'hold': 0}
trend_symbols = {'down': 0, 'up': 0, 'no change': 0}

for i in range(total_trend):
    trend_symbols['down'] += 1 if discrete_trend_data[i] == 0 else 0
    trend_symbols['up'] += 1 if discrete_trend_data[i] == 1 else 0
    trend_symbols['no change'] += 1 if discrete_trend_data[i] == 2 else 0

for i in range(total_price):
    price_symbols['sell'] += 1 if discrete_price_data[i] == 0 else 0
    price_symbols['buy'] += 1 if discrete_price_data[i] == 1 else 0
    price_symbols['hold'] += 1 if discrete_price_data[i] == 2 else 0

hidden_states = []
for trend_value, price_value in zip(discrete_trend_data, discrete_price_data):
    if (trend_value == 1 and price_value == 1) or (trend_value == 1 and price_value == 2):  # Trend indicates buy opportunity
        hidden_states.append("buy")
    elif (trend_value == 0 and price_value == 0) or (trend_value == 0 and price_value == 2):  # Trend indicates sell opportunity
        hidden_states.append("sell")
    else:  # Trend and price do not provide clear buy or sell signals
        hidden_states.append("hold")



In [105]:
# Step 2: Counting Observations
# Initialize dictionaries to store symbol counts for each hidden state
trend_symbol_counts = {}
price_symbol_counts = {}
state_counts = {}

# Iterate over the time series data of trend and price data along with the corresponding hidden states
for state, trend_symbol, price_symbol in zip(hidden_states, discrete_trend_data, discrete_price_data):
    # Update symbol counts for trend data
    if state not in trend_symbol_counts:
        trend_symbol_counts[state] = {}
    if trend_symbol not in trend_symbol_counts[state]:
        trend_symbol_counts[state][trend_symbol] = 0
    trend_symbol_counts[state][trend_symbol] += 1
    
    # Update symbol counts for price data
    if state not in price_symbol_counts:
        price_symbol_counts[state] = {}
    if price_symbol not in price_symbol_counts[state]:
        price_symbol_counts[state][price_symbol] = 0
    price_symbol_counts[state][price_symbol] += 1
    
    # Update state counts
    if state not in state_counts:
        state_counts[state] = 0
    state_counts[state] += 1

# Step 3: Calculating Probabilities
# Calculate emission probabilities for trend data
trend_emission_probs = {}
for state in trend_symbol_counts:
    trend_emission_probs[state] = {}
    total_count = state_counts[state]
    for trend_symbol, count in trend_symbol_counts[state].items():
        trend_emission_probs[state][trend_symbol] = count / total_count

# Calculate emission probabilities for price data
price_emission_probs = {}
for state in price_symbol_counts:
    price_emission_probs[state] = {}
    total_count = state_counts[state]
    for price_symbol, count in price_symbol_counts[state].items():
        price_emission_probs[state][price_symbol] = count / total_count

# Step 4: Constructing the Transition Matrix
# Construct count of transitions from one hidden state to another i.e how many times 0 becomes 1, 1 becomes 2 etc.
transition_counts = {}
for i in range(1, len(hidden_states)):
    transition = (hidden_states[i-1], hidden_states[i])
    if transition not in transition_counts:
        transition_counts[transition] = 0
    transition_counts[transition] += 1

# Construct the transition matrix
transition_matrix = np.zeros((3, 3))
for (state1, state2), count in transition_counts.items():
    state1_index = {"buy": 0, "hold": 1, "sell": 2}[state1]
    state2_index = {"buy": 0, "hold": 1, "sell": 2}[state2]
    transition_matrix[state1_index, state2_index] = count / state_counts[state1]



# Step 5: Constructing the Emission Matrices
# Each row corresponds to a hidden state, and each column corresponds to a discrete symbol
trend_emission_matrix = np.array([[trend_emission_probs[state].get(symbol, 0) for symbol in set(discrete_trend_data)] for state in trend_emission_probs.keys()])
price_emission_matrix = np.array([[price_emission_probs[state].get(symbol, 0) for symbol in set(discrete_price_data)] for state in price_emission_probs.keys()])

# row = hidden state (s,b,h), column = observation symbol (down,up,no change)

# Example printing transition matrix
print("Transition Matrix:")
print(transition_matrix)

# Example printing emission matrices
print("Trend Emission Matrix:")
print(trend_emission_matrix)

print("Price Emission Matrix:")
print(price_emission_matrix)

Transition Matrix:
[[0.18032787 0.55737705 0.24590164]
 [0.26277372 0.5620438  0.17518248]
 [0.31818182 0.56818182 0.11363636]]
Trend Emission Matrix:
[[0.2189781  0.05109489 0.72992701]
 [1.         0.         0.        ]
 [0.         1.         0.        ]]
Price Emission Matrix:
[[0.24817518 0.49635036 0.25547445]
 [0.38636364 0.         0.61363636]
 [0.         0.63934426 0.36065574]]
